# Compare Implementation of DML IRM in Causalis and DML IRM in DoubleML

Comparing `IRM` model from Causalis with `dml.DoubleMLIRM` from DoubleML with default CatboostRegressor and CatboostClassifier for g0, g1 amd m

# DGP

We will use DGP: `generate_obs_hte_26_rich()`
read more at [this notebook](https://causalis.causalcraft.com/articles/generate_obs_hte_26_rich)

In [1]:
from causalis.scenarios.unconfoundedness.dgp import generate_obs_hte_26_rich

data = generate_obs_hte_26_rich(return_causal_data=False, include_oracle=True)
data.head()


,y,d,tenure_months,avg_sessions_week,spend_last_month,age_years,income_monthly,prior_purchases_12m,support_tickets_90d,premium_user,mobile_user,urban_resident,referred_user,m,m_obs,tau_link,g0,g1,cate
0,0.000000,0.0,28.814654,1.0,77.936767,50.234101,1926.698301,1.0,2.0,1.0,1.0,1.0,0.0,0.047970,0.047970,1.330764,8.137981,35.177086,27.039105
1,559.364158,1.0,25.913345,3.0,53.777740,28.115859,5104.271509,3.0,0.0,1.0,1.0,0.0,1.0,0.049695,0.049695,2.190209,60.459257,584.580685,524.121427
2,26.143003,1.0,24.969929,10.0,134.764322,22.907062,5267.938255,8.0,3.0,0.0,1.0,1.0,0.0,0.077087,0.077087,1.570177,7.712855,38.297992,30.585137
3,19.283585,1.0,40.655089,5.0,59.517074,31.970490,6597.327018,3.0,2.0,1.0,1.0,1.0,0.0,0.069481,0.069481,1.933844,25.386510,189.737828,164.351318
4,0.000000,1.0,18.560899,3.0,74.370930,39.237248,4930.009628,5.0,1.0,1.0,1.0,0.0,0.0,0.047097,0.047097,1.818265,15.359250,102.433597,87.074347


In [13]:
print(f"Ground truth ATTE is {data[data['d'] == 1]['cate'].mean()}")

Ground truth ATTE is 837.4043605736649


In [3]:
from causalis.data_contracts import CausalData

causaldata = CausalData(df = data,
                        treatment='d',
                        outcome='y',
                        confounders=['tenure_months', 'avg_sessions_week', 'spend_last_month', 'age_years', 'income_monthly', 'prior_purchases_12m', 'support_tickets_90d', 'premium_user', 'mobile_user', 'urban_resident', 'referred_user'])
causaldata

CausalData(df=(100000, 13), treatment='d', outcome='y', confounders=['tenure_months', 'avg_sessions_week', 'spend_last_month', 'age_years', 'income_monthly', 'prior_purchases_12m', 'support_tickets_90d', 'premium_user', 'mobile_user', 'urban_resident', 'referred_user'])

# Comparison of Inference

## Causalis

In [4]:
from causalis.scenarios.unconfoundedness import IRM

model = IRM().fit(causaldata)

In [5]:
result_causalis = model.estimate(score='ATTE')
result_causalis.summary()

,estimand,coefficient,p_val,lower_ci,upper_ci,relative_diff_%,is_significant
0,ATTE,817.128619,0.0,749.253009,885.00423,893.9941,True


## DoubleML

In [6]:
import doubleml as dml
data_dml_base = dml.DoubleMLData(data,
                                 y_col='y',
                                 d_cols='d',
                                 x_cols=['tenure_months', 'avg_sessions_week', 'spend_last_month', 'age_years', 'income_monthly', 'prior_purchases_12m', 'support_tickets_90d', 'premium_user', 'mobile_user', 'urban_resident', 'referred_user'])

In [7]:
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from catboost import CatBoostRegressor, CatBoostClassifier
import numpy as np

class SkCatBoostRegressor(RegressorMixin, BaseEstimator):
    def __init__(self, **params):
        self.params = params
        self.model_ = CatBoostRegressor(**params)

    def fit(self, X, y, **fit_params):
        self.model_ = CatBoostRegressor(**self.params)
        self.model_.fit(X, y, verbose=False, **fit_params)
        try:
            self.n_features_in_ = X.shape[1]
        except Exception:
            pass
        return self

    def predict(self, X):
        return self.model_.predict(X)

    def get_params(self, deep=True):
        return dict(self.params)

    def set_params(self, **params):
        self.params.update(params)
        self.model_ = CatBoostRegressor(**self.params)
        return self


class SkCatBoostClassifier(ClassifierMixin, BaseEstimator):
    def __init__(self, **params):
        self.params = params
        self.model_ = CatBoostClassifier(**params)

    def fit(self, X, y, **fit_params):
        self.model_ = CatBoostClassifier(**self.params)
        self.model_.fit(X, y, verbose=False, **fit_params)
        if hasattr(self.model_, "classes_"):
            self.classes_ = self.model_.classes_
        else:
            self.classes_ = np.unique(y)
        try:
            self.n_features_in_ = X.shape[1]
        except Exception:
            pass
        return self

    def predict(self, X):
        return self.model_.predict(X)

    def predict_proba(self, X):
        proba = self.model_.predict_proba(X)
        if hasattr(self.model_, "classes_") and list(self.model_.classes_) != list(self.classes_):
            order = [list(self.model_.classes_).index(c) for c in self.classes_]
            proba = np.asarray(proba)[:, order]
        return proba

    def get_params(self, deep=True):
        return dict(self.params)

    def set_params(self, **params):
        self.params.update(params)
        self.model_ = CatBoostClassifier(**self.params)
        return self

In [8]:
boost = SkCatBoostRegressor(iterations=500, depth=6, learning_rate=0.1)
boost_class = SkCatBoostClassifier(iterations=500, depth=6, learning_rate=0.1)

In [14]:
result_doubleml = dml.DoubleMLIRM(
    data_dml_base,
    ml_g=boost,
    ml_m=boost_class,
    trimming_threshold=0.01,
    n_folds=3,
    score='ATTE'
)
result_doubleml.fit()
result_doubleml.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
d,816.580926,34.612272,23.592237,4.630412e-123,748.742119,884.419733


# Conclusion

In [11]:
result_causalis.summary()

,estimand,coefficient,p_val,lower_ci,upper_ci,relative_diff_%,is_significant
0,ATTE,817.128619,0.0,749.253009,885.00423,893.9941,True


In [12]:
result_doubleml.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
d,817.651251,34.638608,23.605199,3.408269e-123,749.760827,885.541676


Results are very close